# Utiliser la fonction pivot sur un DF
Création d'une fonction exploitant la table pivot et permettant de récupérer des données dites liées.

In [8]:
import pandas as pd
import numpy as np

In [2]:
# Création d'un dataframe
# Dans l'exemple, on remarque que les colonnes rens et A sont totalement liées.
col1 = ['a','c', 'a','b' ]
col2 = ['na','k','cl','k']
col3=  [5,6,7,8]
col4= ["rens1", "rens3", "rens1", 'rens2']

df = pd.DataFrame({'A' : col1, 'B': col2, 'C': col3, 'rens': col4})

In [3]:
df

,A,B,C,rens
0,a,na,5,rens1
1,c,k,6,rens3
2,a,cl,7,rens1
3,b,k,8,rens2


In [4]:
df.pivot(index = 'A', columns='B', values='C')

B,cl,k,na
A,,,
a,7.0,NaN,5.0
b,NaN,8.0,NaN
c,NaN,6.0,NaN


Dans la table ci-dessus, les données liées ne sont pas rapportées. Comme je n'ai pas trouvé  comment les récupérer avec la fonction pivot, j'ai créé une fonction.

In [6]:
RENS = df.reindex(df.A.drop_duplicates().index)[['A','rens']]
RENS

,A,rens
0,a,rens1
1,c,rens3
3,b,rens2


In [7]:
PIVOT = df.pivot(index = 'A', columns='B', values='C')
PIVOT

B,cl,k,na
A,,,
a,7.0,NaN,5.0
b,NaN,8.0,NaN
c,NaN,6.0,NaN


In [8]:
# et on fait un merge en PIVOT et RENS

In [9]:
PIVOT.merge(RENS, left_on = PIVOT.index, right_on= 'A' , how = 'left')

,cl,k,na,A,rens
0,7.0,NaN,5.0,a,rens1
1,NaN,8.0,NaN,b,rens2
2,NaN,6.0,NaN,c,rens3


# Fonction pour automatiser tout le bazard !

In [11]:
def to_list(a):
    """return [str] or a list
    a: str or list
    >>> to_list("foo")
    ['foo']
    >>> to_list(['foo', 'bar'])
    ['foo', 'bar']
    """
    
    if isinstance(a, (str, int)):
        return [a]
    elif isinstance(a,list):
        return a

def pivot_bm(df, index_col, cat_col, val_col,  bound_col, verbose = False):
    """
    index_col : colonne l'identifiant de l'examen
    cat_col : nom de l'examen (exemple : "sodium")
    val_col : valeur d examen(exemple : 135)
    bound_col : (exemple : "enceinte" ou ["enceinte", "hospitalisée"])
    """ 
    # Je fabrique la lsite de toutes les colonnes à ajouter.
    lst_rens_col = [ index_col]
    lst_rens_col.extend(to_list(bound_col))
    if verbose : 
        print("lst_rens_col=",lst_rens_col)  ; 
    RENS = df.reindex(df[index_col].drop_duplicates().index)[lst_rens_col]
    if verbose :
        print("\nRENS :") ;
        print(RENS) 

    PIVOT = df.pivot(index = index_col, columns=cat_col, values=val_col)
    if verbose :
        print("\nPIVOT")
        print(PIVOT)
    
    return PIVOT.merge(RENS, left_on = PIVOT.index, right_on= index_col , how = 'left')
import doctest
doctest.testmod()

TestResults(failed=0, attempted=2)

# Application

In [15]:
# Création d'un dataframe
# Dans l'exemple, on remarque que les colonnes rens et A sont totalement liées.
col1 = ['a','c', 'a','b' ]
col2 = ['na','k','cl','k']
col3 = [5,6,7,8]
col4 = ["rens1", "rens3", "rens1", 'rens2']
col5 = ["rens4", 'rens5', 'rens4', 'rens7']

df = pd.DataFrame({'A' : col1, 'B': col2, 'C': col3, 'rens': col4})
df2 = pd.DataFrame({'A' : col1, 'B': col2, 'C': col3, 'rens': col4, "rens2" : col5})


In [18]:
# Exemple avec 1 colonne liée
df

,A,B,C,rens
0,a,na,5,rens1
1,c,k,6,rens3
2,a,cl,7,rens1
3,b,k,8,rens2


In [19]:

pivot_bm(df, index_col='A', val_col= 'C', cat_col= 'B', bound_col='rens'  )

,cl,k,na,A,rens
0,7.0,NaN,5.0,a,rens1
1,NaN,8.0,NaN,b,rens2
2,NaN,6.0,NaN,c,rens3


In [14]:
# Exemple avec plusieurs colonnes liées.
pivot_bm(df2, index_col='A', val_col= 'C', cat_col= 'B', bound_col=['rens', 'rens2']  )

,cl,k,na,A,rens,rens2
0,7.0,NaN,5.0,a,rens1,rens4
1,NaN,8.0,NaN,b,rens2,rens7
2,NaN,6.0,NaN,c,rens3,rens5
